In [1]:
import json
import re
import time
import requests
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
from datetime import datetime
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')
from random import choice

In [2]:
current_datetime = datetime.now().strftime("%d-%m-%Y")
month_now = datetime.now().month
day_now = datetime.now().day

In [3]:
way = os.getcwd()

# Cookies and Headers для парсинга скидки и цены

In [4]:
cookies_discount = {
    'tmr_lvid': '286726132090325c59b36aa6b5e4aeaa',
    'tmr_lvidTS': '1692163620235',
    '_ym_uid': '1692163620496112993',
    '_ym_d': '1692163620',
    'city_id': '118',
    'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
    'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_subdomain': 'spb',
    '_tt_enable_cookie': '1',
    '_ttp': 'tbKEEA3IkGrgdKpaT6-HRNf0K8h',
    'ccart': 'off',
    'uxs_uid': '898ab7e0-3bf5-11ee-8efe-f5b770db12a1',
    'loggedin': '%D0%AE%D1%80%D0%B8%D0%B9',
    'cseid': 'f4935df6bdac0029832e99cef94a5bc2',
    'sessionid': 'nr24mlml3ze25rk734gkkmgx3yv9ou7x',
    'authorization': 'true',
    'mindboxDeviceUUID': 'c254fa3f-cfcd-4146-9778-65be21c6a4df',
    'directCrm-session': '%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D',
    'auid': '651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA',
    '_ga_G74SVHM1ZG': 'GS1.2.1700824600.19.0.1700824600.60.0.0',
    '_ga_GRDJMMMRHQ': 'GS1.2.1700824600.33.0.1700824601.59.0.0',
    '_ga_GRSMPDQ6SE': 'GS1.2.1700824598.88.1.1700825516.60.0.0',
    '_ga_M9ZHPEJXBM': 'GS1.2.1700824599.88.1.1700825517.60.0.0',
    'qr-go-to-the-app-banner': '1',
    'sncid': 'b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams',
    'clt': 'new_old',
    '_ym_isad': '2',
    '_gid': 'GA1.2.2131018395.1702274408',
    'city_auto_popup_shown': '1',
    '_ym_visorc': 'b',
    '_ga': 'GA1.2.333284150.1692163620',
    '_ga_HJNSJ6NG5J': 'GS1.1.1702276831.120.1.1702280271.20.0.0',
    'digi_uc': 'W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
}

headers_discount = {
    'AB-TESTS': '{}',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': 'tmr_lvid=286726132090325c59b36aa6b5e4aeaa; tmr_lvidTS=1692163620235; _ym_uid=1692163620496112993; _ym_d=1692163620; city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; _tt_enable_cookie=1; _ttp=tbKEEA3IkGrgdKpaT6-HRNf0K8h; ccart=off; uxs_uid=898ab7e0-3bf5-11ee-8efe-f5b770db12a1; loggedin=%D0%AE%D1%80%D0%B8%D0%B9; cseid=f4935df6bdac0029832e99cef94a5bc2; sessionid=nr24mlml3ze25rk734gkkmgx3yv9ou7x; authorization=true; mindboxDeviceUUID=c254fa3f-cfcd-4146-9778-65be21c6a4df; directCrm-session=%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D; auid=651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA; _ga_G74SVHM1ZG=GS1.2.1700824600.19.0.1700824600.60.0.0; _ga_GRDJMMMRHQ=GS1.2.1700824600.33.0.1700824601.59.0.0; _ga_GRSMPDQ6SE=GS1.2.1700824598.88.1.1700825516.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1700824599.88.1.1700825517.60.0.0; qr-go-to-the-app-banner=1; sncid=b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams; clt=new_old; _ym_isad=2; _gid=GA1.2.2131018395.1702274408; city_auto_popup_shown=1; _ym_visorc=b; _ga=GA1.2.333284150.1692163620; _ga_HJNSJ6NG5J=GS1.1.1702276831.120.1.1702280271.20.0.0; digi_uc=W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
    'Origin': 'https://spb.sunlight.net',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'SunlightFrontendApp',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# params = {
#     'article': '35517',
#     'city_id': '118',
# }

# response = requests.get(
#     'https://api.sunlight.net/v6/catalog/products/card/purchase/',
#     params=params,
#     cookies=cookies_discount,
#     headers=headers_discount,
# )

# Сбор скидок и цен

In [5]:
def get_personal_price(table):
    data_price = []
    for art in tqdm(range(len(table['article']))):
        # prx = choice(proxy_list)
        params = {
            'article': table['article'][art],
            'city_id': '118'
        }
        try:
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
        )
        except:
            time.sleep(6)
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        try:
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        except:
            time.sleep(6)
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        discount = []
        try:
            price_old = price_json.json()['content']['default_price']['base_sum']
        except:
            price_old = np.nan
        try:
            price = price_json.json()['content']['default_price'][('result_sum')]
        except:
            price = np.nan
        try:
            price_personal = price_json.json()['content']['default_price']['personal_block']['value']
        except:
            price_personal = np.nan

        try:
            for i in range(len(discount_json.json()['content']['nameplates'])):
                discount.append(discount_json.json()['content']['nameplates'][i]['text']['label'])
        except:
            discount.append(np.nan)
            
        data_price.append({
            'article': table['article'][art],
            'price_old': price_old,
            'price': price,
            'price_personal': price_personal,
            'discount': discount

        })
    return data_price

# Сбор цены, названия, группы, артикула

In [6]:
def get_sunlight(soup):
    try:
        price = soup.find(class_="supreme-product-card__price-discount-price").text
        price = re.sub(r'\D', '', str(price))
    except:
        price = np.nan
    try:
        name_product = soup.find('h1', class_="supreme-product-card__about-title").text
    except:
        name_product = np.nan
    try:
        group_product = soup.find_all('span', {'property': 'name'})[1].text
    except:
        group_product = np.nan
    try:
        article_product = soup.find_all('div', class_="supreme-product-card__product-article")[0].find('span').text
        article_product = re.sub('\D', '', str(article_product))
    except:
        article_product = np.nan
    return price, name_product, group_product, article_product

# Сбор всех параметров словарем

In [7]:
def get_params(soup):
    names = []
    values = []
    leight = len(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'}))
    list_names = soup.find_all('div', {'class': 'supreme-product-card-description__item-title'})[-leight:]
    for i in range(leight):
        try:
            nam = list_names[i].text
            names.append(nam.replace('\n', '').strip())
        except:
            names.append(np.nan)
        try:
            values.append(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'})[i].text)
        except:
            values.append(np.nan)
    sunlight_params = dict(zip(names, values))
    return sunlight_params

# Сбор ссылок

In [8]:
def get_links(url_sunlight):
    home = 'https://sunlight.net'
    links = []
    pages_max = 250
    pages = int(pages_max + 1)
    for i in tqdm(range(1, pages)):
        url = f'{url_sunlight}&page={i}'
        resp = requests.get(url, headers={'User-Agent': UserAgent().chrome}, verify=False, timeout=70)
        soup = BeautifulSoup(resp.text, 'html.parser')
        for link in soup.find_all('div', class_="cl-item js-cl-item"):
            links.append(home + link.find('a', href=True)['href'])
    links = list(map(lambda x: x.replace('#product-reviews', ''), links))
    return links

# Общий сбор данных

In [9]:
def sunlight_parser(links):
    data = []
    click = 0
    for link_pars in tqdm(links):
        # prx = choice(proxy_list)
        try:
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        except:
            time.sleep(6)
            resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        good = BeautifulSoup(resp.text, 'lxml')
        data.append({"date": current_datetime,
                    "competitor": "Sunlight",
                    "name": get_sunlight(good)[1],
                    "group": get_sunlight(good)[2],
                    "params": get_params(good),
                    "price": get_sunlight(good)[0],
                    "article": get_sunlight(good)[3],
                    'url': link_pars
                    })
        if click == 50:
            click = 0
            time.sleep(0.5)
        else:
            continue
    data = pd.DataFrame(data)
    return data

In [10]:
links_sunlight = get_links('https://spb.sunlight.net/catalog/?proba=2494')

  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [04:29<00:00,  1.08s/it]


In [11]:
b = pd.DataFrame(links_sunlight)

In [12]:
b = b.drop_duplicates()

In [13]:
b = b[0].tolist()

In [14]:
slices = 100
chunks = np.array_split(b, slices)

In [15]:
n = 0
for i in range(n, slices):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = sunlight_parser(chunks[i])
    df.to_csv(rf'{way}\parsed_data\sunlight_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 140/140 [02:14<00:00,  1.04it/s]


начало парсинга 2 части ссылок


100%|██████████| 140/140 [02:08<00:00,  1.09it/s]


начало парсинга 3 части ссылок


...

100%|██████████| 139/139 [02:11<00:00,  1.06it/s]


начало парсинга 99 части ссылок


100%|██████████| 139/139 [01:51<00:00,  1.24it/s]


начало парсинга 100 части ссылок


100%|██████████| 139/139 [01:54<00:00,  1.21it/s]


In [16]:
os.chdir(rf'{way}\parsed_data')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

100

In [17]:
data = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data = pd.concat([df, data], ignore_index=True)

In [18]:
data

,date,competitor,name,group,params,price,article,url
0,13-06-2024,Sunlight,\n Золотой браслет на ногу\n,Браслеты,"{'Модель': '7025051Р-26*', 'Материал изделия':...",7528,988818,https://sunlight.net/catalog/bracelets_988818....
1,13-06-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '01Л161615*', 'Материал изделия': '...",316060,988808,https://sunlight.net/catalog/neck_decoration_9...
2,13-06-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '01Б730073*', 'Материал изделия': '...",71640,988802,https://sunlight.net/catalog/bracelets_988802....
3,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7703861/19,5*', 'Вставка': 'Фианит...",14520,345882,https://sunlight.net/catalog/bracelets_345882....
4,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7700406*', 'Материал изделия': 'Ро...",78290,345855,https://sunlight.net/catalog/bracelets_345855....
...,...,...,...,...,...,...,...,...
13930,13-06-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '3101-K5Y-04', 'Материал изделия': ...",6990,152015,https://sunlight.net/catalog/ring_152015.html
13931,13-06-2024,Sunlight,\n Золотые серьги с агатом и бриллиантами\n,Серьги,"{'Модель': '71794-C5R-01', 'Материал изделия':...",97990,310221,https://sunlight.net/catalog/earring_310221.html
13932,13-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '30906.1*', 'Материал изделия': 'Ро...",5490,91078,https://sunlight.net/catalog/earring_91078.html
13933,13-06-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '12623.1*', 'Материал изделия': 'Бе...",9990,88213,https://sunlight.net/catalog/pendants_88213.html


In [19]:
data.to_csv(rf'{way}\data_without_price.csv', index = False)

In [20]:
# data = pd.read_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\data_without_price.csv')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13935 entries, 0 to 13934
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        13935 non-null  object
 1   competitor  13935 non-null  object
 2   name        13935 non-null  object
 3   group       13935 non-null  object
 4   params      13935 non-null  object
 5   price       13935 non-null  int64 
 6   article     13935 non-null  int64 
 7   url         13935 non-null  object
dtypes: int64(2), object(6)
memory usage: 871.1+ KB


In [22]:
data.duplicated().sum()

0

In [23]:
data = data.drop_duplicates()

In [24]:
data.duplicated().sum()

0

In [25]:
data['article'].isna().sum()

0

In [26]:
data = data.dropna()

In [27]:
data['article'] = data['article'].astype('int') 

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13935 entries, 0 to 13934
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        13935 non-null  object
 1   competitor  13935 non-null  object
 2   name        13935 non-null  object
 3   group       13935 non-null  object
 4   params      13935 non-null  object
 5   price       13935 non-null  int64 
 6   article     13935 non-null  int32 
 7   url         13935 non-null  object
dtypes: int32(1), int64(1), object(6)
memory usage: 816.6+ KB


In [28]:
test = np.array_split(data, 20)

In [36]:
test

[           date competitor                                               name  \
 0    13-06-2024   Sunlight                  \n    Золотой браслет на ногу\n     
 1    13-06-2024   Sunlight     \n    Золотое шейное украшение с фианитами\n     
 2    13-06-2024   Sunlight                          \n    Золотой браслет\n     
 3    13-06-2024   Sunlight              \n    Золотой браслет с фианитами\n     
 4    13-06-2024   Sunlight              \n    Золотой браслет с фианитами\n     
 ..          ...        ...                                                ...   
 692  13-06-2024   Sunlight                         \n    Золотая подвеска\n     
 693  13-06-2024   Sunlight  \n    Золотые серьги с сапфирами и бриллиантам...   
 694  13-06-2024   Sunlight  \n    Золотые серьги с фианитами и наноситалла...   
 695  13-06-2024   Sunlight            \n    Золотые серьги с бриллиантами\n     
 696  13-06-2024   Sunlight               \n    Золотое кольцо с фианитами\n     
 
              

In [40]:
test[0]

,date,competitor,name,group,params,price,article,url
0,13-06-2024,Sunlight,\n Золотой браслет на ногу\n,Браслеты,"{'Модель': '7025051Р-26*', 'Материал изделия':...",7528,988818,https://sunlight.net/catalog/bracelets_988818....
1,13-06-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '01Л161615*', 'Материал изделия': '...",316060,988808,https://sunlight.net/catalog/neck_decoration_9...
2,13-06-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '01Б730073*', 'Материал изделия': '...",71640,988802,https://sunlight.net/catalog/bracelets_988802....
3,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7703861/19,5*', 'Вставка': 'Фианит...",14520,345882,https://sunlight.net/catalog/bracelets_345882....
4,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7700406*', 'Материал изделия': 'Ро...",78290,345855,https://sunlight.net/catalog/bracelets_345855....
...,...,...,...,...,...,...,...,...
692,13-06-2024,Sunlight,\n Золотая подвеска\n,Подвески,"{'Модель': '097950_03_01_000_0000*', 'Материал...",5990,333334,https://sunlight.net/catalog/pendants_333334.html
693,13-06-2024,Sunlight,\n Золотые серьги с сапфирами и бриллиантам...,Серьги,"{'Модель': '72356-C5R-01', 'Материал изделия':...",109990,333037,https://sunlight.net/catalog/earring_333037.html
694,13-06-2024,Sunlight,\n Золотые серьги с фианитами и наноситалла...,Серьги,"{'Модель': 'кл3585а-48*', 'Материал изделия': ...",40990,332957,https://sunlight.net/catalog/earring_332957.html
695,13-06-2024,Sunlight,\n Золотые серьги с бриллиантами\n,Серьги,"{'Модель': '73118-C5W-01', 'Материал изделия':...",699990,344886,https://sunlight.net/catalog/earring_344886.html


In [42]:
n = 0
for i in range(n, 20):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = get_personal_price(test[i].reset_index())
    df = pd.DataFrame(df)
    df.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts\\' + f'sunlight_all_data_{day_now}_{month_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 697/697 [06:23<00:00,  1.82it/s]


начало парсинга 2 части ссылок


100%|██████████| 697/697 [06:17<00:00,  1.84it/s]


начало парсинга 3 части ссылок


...

100%|██████████| 696/696 [08:50<00:00,  1.31it/s]


начало парсинга 19 части ссылок


100%|██████████| 696/696 [08:57<00:00,  1.29it/s]


начало парсинга 20 части ссылок


100%|██████████| 696/696 [09:33<00:00,  1.21it/s]


In [48]:
os.chdir(r'C:\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
len(all_filenames)

20

In [49]:
data

,date,competitor,name,group,params,price,article,url
0,13-06-2024,Sunlight,\n Золотой браслет на ногу\n,Браслеты,"{'Модель': '7025051Р-26*', 'Материал изделия':...",7528,988818,https://sunlight.net/catalog/bracelets_988818....
1,13-06-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '01Л161615*', 'Материал изделия': '...",316060,988808,https://sunlight.net/catalog/neck_decoration_9...
2,13-06-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '01Б730073*', 'Материал изделия': '...",71640,988802,https://sunlight.net/catalog/bracelets_988802....
3,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7703861/19,5*', 'Вставка': 'Фианит...",14520,345882,https://sunlight.net/catalog/bracelets_345882....
4,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7700406*', 'Материал изделия': 'Ро...",78290,345855,https://sunlight.net/catalog/bracelets_345855....
...,...,...,...,...,...,...,...,...
13930,13-06-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '3101-K5Y-04', 'Материал изделия': ...",6990,152015,https://sunlight.net/catalog/ring_152015.html
13931,13-06-2024,Sunlight,\n Золотые серьги с агатом и бриллиантами\n,Серьги,"{'Модель': '71794-C5R-01', 'Материал изделия':...",97990,310221,https://sunlight.net/catalog/earring_310221.html
13932,13-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '30906.1*', 'Материал изделия': 'Ро...",5490,91078,https://sunlight.net/catalog/earring_91078.html
13933,13-06-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '12623.1*', 'Материал изделия': 'Бе...",9990,88213,https://sunlight.net/catalog/pendants_88213.html


In [50]:
data_discount = pd.DataFrame()
for name  in all_filenames:
    df = pd.read_csv(name)
    data_discount = pd.concat([df, data_discount], ignore_index=True)

In [51]:
data_discount

,article,price_old,price,price_personal,discount
0,327366,416 633 ₽,124 990 ₽,96 990 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
1,348026,37 980 ₽,18 990 ₽,11 394 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
2,986913,95 980 ₽,47 990 ₽,28 794 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']"
3,45746,43 980 ₽,21 990 ₽,13 194 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
4,130354,81 480 ₽,40 740 ₽,24 444 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
...,...,...,...,...,...
13930,333334,11 980 ₽,5 990 ₽,3 594 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
13931,333037,219 980 ₽,109 990 ₽,81 990 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']"
13932,332957,81 980 ₽,40 990 ₽,24 594 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']"
13933,344886,NaN,699 990 ₽,671 990 ₽,[nan]


In [52]:
data = data.merge(data_discount, on='article', how='left')

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13935 entries, 0 to 13934
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            13935 non-null  object
 1   competitor      13935 non-null  object
 2   name            13935 non-null  object
 3   group           13935 non-null  object
 4   params          13935 non-null  object
 5   price_x         13935 non-null  int64 
 6   article         13935 non-null  int32 
 7   url             13935 non-null  object
 8   price_old       13097 non-null  object
 9   price_y         13935 non-null  object
 10  price_personal  13212 non-null  object
 11  discount        13935 non-null  object
dtypes: int32(1), int64(1), object(10)
memory usage: 1.2+ MB


In [54]:
data['discount'].value_counts()

discount
['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']                                                  9357
['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']                                                  3455
['ХИТ-цена']                                                                                     710
['Цены снижены! Ещё -15%, код: СЕГОДНЯ']                                                         198
[nan]                                                                                            134
['Цены 8 марта! Ещё -35%, код: МАРТ до 23:59']                                                    28
['Цены 8 марта! Ещё -20%, код: МАРТ до 23:59']                                                    19
['Цены 8 марта! Ещё -30%, код: МАРТ до 23:59']                                                    10
['Цены 8 марта! Ещё -45%, код: МАРТ до 23:59']                                                     9
['Цены 8 марта! Ещё -55%, код: МАРТ до 23:59']                                    

In [55]:
data.head(20)

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,13-06-2024,Sunlight,\n Золотой браслет на ногу\n,Браслеты,"{'Модель': '7025051Р-26*', 'Материал изделия':...",7528,988818,https://sunlight.net/catalog/bracelets_988818....,NaN,7 528 ₽,NaN,['ХИТ-цена']
1,13-06-2024,Sunlight,\n Золотое шейное украшение с фианитами\n,Шейное украшение,"{'Модель': '01Л161615*', 'Материал изделия': '...",316060,988808,https://sunlight.net/catalog/neck_decoration_9...,632 120 ₽,316 060 ₽,288 060 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']"
2,13-06-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '01Б730073*', 'Материал изделия': '...",71640,988802,https://sunlight.net/catalog/bracelets_988802....,143 280 ₽,71 640 ₽,43 640 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
3,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7703861/19,5*', 'Вставка': 'Фианит...",14520,345882,https://sunlight.net/catalog/bracelets_345882....,29 040 ₽,14 520 ₽,8 712 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
4,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7700406*', 'Материал изделия': 'Ро...",78290,345855,https://sunlight.net/catalog/bracelets_345855....,156 580 ₽,78 290 ₽,50 290 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
5,13-06-2024,Sunlight,\n Золотой браслет с фианитами\n,Браслеты,"{'Модель': '7700377*', 'Материал изделия': 'Ро...",94480,345853,https://sunlight.net/catalog/bracelets_345853....,188 960 ₽,94 480 ₽,66 480 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
6,13-06-2024,Sunlight,\n Золотой браслет с изумрудами и бриллиант...,Браслеты,"{'Модель': '73963-B5W-02', 'Материал изделия':...",129990,987833,https://sunlight.net/catalog/bracelets_987833....,259 980 ₽,129 990 ₽,101 990 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']"
7,13-06-2024,Sunlight,\n Золотой браслет с сапфирами и бриллианта...,Браслеты,"{'Модель': '73963-B5W-01', 'Материал изделия':...",114990,987832,https://sunlight.net/catalog/bracelets_987832....,229 980 ₽,114 990 ₽,86 990 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']"
8,13-06-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '46-0010-11*', 'Материал изделия': ...",88830,345090,https://sunlight.net/catalog/bracelets_345090....,177 660 ₽,88 830 ₽,60 830 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"
9,13-06-2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '46-0009-11-61*', 'Материал изделия...",81010,345089,https://sunlight.net/catalog/bracelets_345089....,162 020 ₽,81 010 ₽,53 010 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"


In [56]:
data.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\parsers\workspace\\' + f'sunlight_all_{day_now}_{month_now}_{datetime.today().year}.csv', index=False)

In [57]:
previous = pd.read_csv(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_raw.csv')

In [58]:
data = pd.concat([previous, data], ignore_index=True)

In [59]:
data

,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,21-05-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '71409-K5W-02', 'Материал изделия':...",99990.0,985936,https://sunlight.net/catalog/ring_985936.html,199 980 ₽,99 990 ₽,82 990 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
1,21-05-2024,Sunlight,\n Золотой браслет с бриллиантами\n,Браслеты,"{'Модель': '73353-B5R-01', 'Материал изделия':...",399990.0,985849,https://sunlight.net/catalog/bracelets_985849....,799 980 ₽,399 990 ₽,382 990 ₽,"['Второе бесплатно, код:1=2 до 27.05']"
2,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '01Ц032974Д*', 'Материал изделия': ...",133820.0,343470,https://sunlight.net/catalog/chains_343470.html,267 640 ₽,133 820 ₽,116 820 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
3,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '01Ц012974Д*', 'Материал изделия': ...",148990.0,343469,https://sunlight.net/catalog/chains_343469.html,297 980 ₽,148 990 ₽,131 990 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
4,21-05-2024,Sunlight,\n Золотая цепь\n,Цепи,"{'Модель': '45-0029-10*', 'Материал изделия': ...",106200.0,343447,https://sunlight.net/catalog/chains_343447.html,212 400 ₽,106 200 ₽,89 200 ₽,['Ещё -35% по коду: ЗОЛОТО до 23:59']
...,...,...,...,...,...,...,...,...,...,...,...,...
71722,13-06-2024,Sunlight,\n Золотое кольцо с бриллиантами\n,Кольца,"{'Модель': '3101-K5Y-04', 'Материал изделия': ...",6990.0,152015,https://sunlight.net/catalog/ring_152015.html,NaN,6 990 ₽,NaN,['ХИТ-цена']
71723,13-06-2024,Sunlight,\n Золотые серьги с агатом и бриллиантами\n,Серьги,"{'Модель': '71794-C5R-01', 'Материал изделия':...",97990.0,310221,https://sunlight.net/catalog/earring_310221.html,195 980 ₽,97 990 ₽,69 990 ₽,"['Цены 8 марта! Ещё -15%, код: МАРТ до 23:59']"
71724,13-06-2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '30906.1*', 'Материал изделия': 'Ро...",5490.0,91078,https://sunlight.net/catalog/earring_91078.html,NaN,5 490 ₽,NaN,['ХИТ-цена']
71725,13-06-2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '12623.1*', 'Материал изделия': 'Бе...",9990.0,88213,https://sunlight.net/catalog/pendants_88213.html,19 980 ₽,9 990 ₽,5 994 ₽,"['Цены 8 марта! Ещё -10%, код: МАРТ до 23:59']"


In [60]:
data.to_csv(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\parsed_data\sunlight_raw.csv', index=False)

In [61]:
def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f'Ошибка при удалении файла {file_path}. {e}')

In [62]:
delete_files_in_folder('C:\\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data')
delete_files_in_folder('C:\\Users\Trenkin.Sergey\Desktop\parsers\sunlight_current_data\discounts')